In [16]:
from generate_signal import generate_X_matrix
from beamforming import beamforming_method
from generate_signal import generate_A_matrix
from generate_signal import generate_noise
from generate_signal import generate_R_hat
from generate_signal import generate_R_hat_with_phase
from music import music_method
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch


nbSources = 2 # Nombre de sources
nbSensors = 10 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = 10
T = 100
correlation_List = [0.4] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.


In [17]:
import numpy as np
from sklearn.model_selection import train_test_split

phi_max = 90  # Définition de phi_max
rho = 1  # Résolution de la grille
G = np.arange(-phi_max, phi_max + rho, rho)  # Création de la grille

def angles_to_binary_vector(thetaList, G):
    """
    Convertit une liste d'angles en un vecteur binaire représentant la présence/absence d'angles dans la grille G.
    
    Args:
    - thetaList: Liste des angles d'arrivée pour un échantillon.
    - G: Grille des angles possibles.
    
    Returns:
    - Vecteur binaire des labels Z.
    """
    Z = np.zeros(len(G), dtype=int)
    for theta in thetaList:
        index = np.searchsorted(G, theta)  # Trouver l'indice de l'angle dans la grille
        if 0 <= index < len(G):  # S'assurer que l'indice est dans la plage valide
            Z[index] = 1
    return Z

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split

# Paramètres pour la simulation
num_samples = 1000  # Nombre d'échantillons à générer
L = 10  # Nombre de capteurs
T = 100  # Nombre de points temporels
phi_max = 90  # φmax
rho = 1  # Résolution

# Grille des angles
G = np.arange(-phi_max, phi_max + rho, rho)

# Initialisation des tableaux pour les données
all_R_hat_with_phase = np.zeros((num_samples, L, L, 3))
all_Z = np.zeros((num_samples, len(G)))

for i in range(num_samples):
    # Générer les angles d'arrivée et la matrice X comme décrit précédemment
    thetaList = [np.random.uniform(-90, 90) for _ in range(2)]  # Exemple de génération d'angles
    X = generate_X_matrix(nbSources, L, T, thetaList, varList, correlation_List, signal_noise_ratio)  # Votre fonction pour générer X, ajoutez les paramètres nécessaires
    
    # Calculer R_hat étendue avec phase
    R_hat_with_phase = generate_R_hat_with_phase(X)
    all_R_hat_with_phase[i] = R_hat_with_phase
    
    # Calculer les labels Z pour les angles générés
    all_Z[i] = angles_to_binary_vector(thetaList, G)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, Z_train, Z_test = train_test_split(all_R_hat_with_phase, all_Z, test_size=0.2, random_state=42)


[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+11]]
[[1.47268298e+11 1.29801233e+11]
 [1.29801233e+11 7.15036782e+

In [19]:
X_train = torch.from_numpy(R_hat_train).to(dtype=torch.float32)
X_test = torch.from_numpy(R_hat_test).to(dtype=torch.float32)
z_train = torch.from_numpy(z_train).to(dtype=torch.float32)
z_test = torch.from_numpy(z_test).to(dtype=torch.float32)

TypeError: expected np.ndarray (got Tensor)

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class DOADataset(Dataset):
    def __init__(self, covariance_matrices, doa_labels):
        """
        Initialisation du Dataset DOA.

        Args:
        - covariance_matrices (np.ndarray): Matrices de covariance étendues avec la phase.
          Forme attendue: (nb_samples, nbSensors, nbSensors, 3) pour les parties réelle, imaginaire, et la phase.
        - doa_labels (np.ndarray): Labels DOA sous forme de vecteurs binaires.
          Forme attendue: (nb_samples, nb_labels), où nb_labels est le nombre de points dans la grille DOA.
        """
        self.covariance_matrices = covariance_matrices
        self.doa_labels = doa_labels

    def __len__(self):
        return len(self.covariance_matrices)

    def __getitem__(self, idx):
        cov_matrix = self.covariance_matrices[idx]
        label = self.doa_labels[idx]

        # Conversion en tenseurs PyTorch
        # Assurez-vous que cov_matrix est déjà un np.ndarray avec la phase incluse
        cov_matrix = torch.from_numpy(cov_matrix).float()
        label = torch.from_numpy(label).float()

        return cov_matrix, label

# Exemple d'utilisation (remplacez R_hat_train/test et angles_train/test par vos données réelles)
train_dataset = DOADataset(X_train, z_train)
test_dataset = DOADataset(X_test, z_test)

# Création des DataLoaders pour l'entraînement et le test
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)



In [13]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

# Note: we don't recommend using the "mps" device for this notebook.

print(f"Using {device} device")

Using cpu device


In [21]:
import torch
import torch.nn as nn

class DOACNN(nn.Module):
    def __init__(self, num_channels=3, num_classes=242):
        super(DOACNN, self).__init__()

        # Couches convolutionnelles 2D
        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=256, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, padding=1)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, padding=1)
        self.relu4 = nn.ReLU()

        # Couches de ReLU
        self.relu5 = nn.ReLU()
        self.relu6 = nn.ReLU()
        self.relu7 = nn.ReLU()
        self.relu8 = nn.ReLU()

        # Couche d'aplatissement
        self.flatten = nn.Flatten()

        # Couches entièrement connectées (FC)
        self.fc1 = nn.Linear(in_features=256 * 30 * 30, out_features=4096)
        self.relu9 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=4096, out_features=2048)
        self.relu10 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=2048, out_features=1024)
        self.relu11 = nn.ReLU()
        self.fc4 = nn.Linear(in_features=1024, out_features=num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)

        x = self.relu5(x)
        x = self.relu6(x)
        x = self.relu7(x)
        x = self.relu8(x)

        x = self.flatten(x)

        x = self.fc1(x)
        x = self.relu9(x)
        x = self.fc2(x)
        x = self.relu10(x)
        x = self.fc3(x)
        x = self.relu11(x)
        x = self.fc4(x)

        return x

# Création d'une instance du modèle
model = DOACNN()


In [23]:
# Définition du modèle

# Fonction de perte et optimiseur
criterion = nn.MSELoss()  # Mean Squared Error Loss pour une tâche de régression
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Optimiseur Adam avec un taux d'apprentissage de 0.001

# Boucle d'entrainement
epochs = 10  # Nombre d'époques pour l'entrainement
for epoch in range(epochs):
    # Forward pass: Calculer les prédictions du modèle
    outputs = model(X_train)
    loss = criterion(outputs, z_train)
    
    # Backward pass et optimisations
    optimizer.zero_grad()  # Effacer les gradients des paramètres du modèle
    loss.backward()        # Calculer les gradients du loss par rapport aux paramètres
    optimizer.step()       # Mettre à jour les paramètres du modèle
    
    # Afficher les informations d'entrainement
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

# Évaluation du modèle
model.eval()
with torch.no_grad():  # Désactiver le calcul de gradient pour l'évaluation
    predicted_angles = model(X_test)
    test_loss = criterion(predicted_angles, z_test)
    print(f'Test Loss: {test_loss.item()}')

    # Calcul du RMSE pour l'évaluation
    rmse = torch.sqrt(test_loss)
    print(f'Test RMSE: {rmse.item()}')

RuntimeError: Given groups=1, weight of size [256, 3, 3, 3], expected input[800, 10, 10, 3] to have 3 channels, but got 10 channels instead

In [ ]:
loss_fn = criterion

In [24]:
print("Dimension de la sortie du modèle (attendue) : ", z_train.shape[1])
print("Dimension de la sortie du modèle (réelle) : ", model.fc4.out_features)

# Test du modèle avec les données simulées
outputs = model(X_train)
print("Dimension de la sortie du modèle (test) : ", outputs.shape)
print("Dimension des étiquettes (test) : ", angles_train.shape)

Dimension de la sortie du modèle (attendue) :  181
Dimension de la sortie du modèle (réelle) :  242


RuntimeError: Given groups=1, weight of size [256, 3, 3, 3], expected input[800, 10, 10, 3] to have 3 channels, but got 10 channels instead

In [18]:
import torch

# Assumant que model, R_hat_train, R_hat_test, angles_train, angles_test sont déjà définis

def print_tensor_shapes():
    print("Shape of R_hat_train:", R_hat_train.shape)
    print("Shape of angles_train:", angles_train.shape)
    print("Shape of R_hat_test:", R_hat_test.shape)
    print("Shape of angles_test:", angles_test.shape)

def test_model_with_single_batch(model, data, labels):
    # Utilisez seulement un petit lot pour tester
    small_batch = data[:10]  # Prendre les 10 premiers exemples
    small_labels = labels[:10]
    model.eval()  # Mettre le modèle en mode évaluation
    with torch.no_grad():
        output = model(small_batch)
        print("Output shape with small batch:", output.shape)
        print("Expected labels shape with small batch:", small_labels.shape)

# Modification de la classe DoACNN pour imprimer les dimensions à chaque étape
class DoACNN_modified(DoACNN):
    def forward(self, x):
        print("Input shape:", x.shape)
        x = F.relu(self.conv1(x))
        print("After conv1 shape:", x.shape)
        # Répétez pour les autres couches convolutives et avant la couche fc1
        x = self.flatten(x)
        print("After flatten shape:", x.shape)
        x = x.view(-1, self.fc1_input_size)  # Ajustez selon la structure de votre modèle
        print("Before entering fc1 shape:", x.shape)
        x = F.relu(self.fc1(x))
        # Suivez avec les autres couches FC si nécessaire
        x = torch.sigmoid(self.fc4(x))
        print("Final output shape:", x.shape)
        return x

# Utilisez ces fonctions pour diagnostiquer
print_tensor_shapes()

model_modified = DoACNN_modified(input_size=10, φmax=φmax, ρ=ρ)
test_model_with_single_batch(model_modified, R_hat_train, angles_train)

Shape of R_hat_train: torch.Size([800, 1, 10, 10])
Shape of angles_train: torch.Size([800, 100])
Shape of R_hat_test: torch.Size([200, 1, 10, 10])
Shape of angles_test: torch.Size([200, 100])
Input shape: torch.Size([10, 1, 10, 10])
After conv1 shape: torch.Size([10, 256, 5, 5])
After flatten shape: torch.Size([10, 6400])
Before entering fc1 shape: torch.Size([250, 256])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (250x4096 and 1024x1799)